In [1]:
import pandas as pd
import os
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import plotly.express as px
import plotly
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
print('Inicializado Rotina json_collect.py.')
os.chdir(sys.path[0])
# os.chdir(os.path.join(os.getcwd(),"json_collect.py"))                  # Modifica o diretório de trabalho, para os caminhos relativos funcionar. (https://stackoverflow.com/questions/1432924/python-change-the-scripts-working-directory-to-the-scripts-own-directory)
# print(os.getcwd())
%run ./json_collect.py                      

Inicializado Rotina json_collect.py.
json_collect executed


In [3]:
# sites: https://dadosabertos.bcb.gov.br/dataset/expectativas-mercado
# Sintase: https://dadosabertos.bcb.gov.br/dataset/expectativas-mercado/resource/d420a704-75a7-4f45-8f4b-0fca813c70f0
# documentação: https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/documentacao

In [4]:
# Relações datas das reunições inseridas manualmente quando não tem previsão
df_selic_data=pd.read_csv('./input/selic_datas.csv',index_col='Reuniao')
# Dados baixados pelo .json
df_selic=pd.read_csv('./output/selic_data_df.csv',index_col='Reuniao')
df_selic=df_selic.drop(columns=['Unnamed: 0','Indicador'])
df_selic['Data']=df_selic.apply(lambda x: datetime.datetime.strptime(x.Data,'%Y-%m-%d').date(),axis=1)

# Relaciona a reunião com sua Data.
df_selic['data_reuniao']=''
for i in range(0, len(df_selic)):
    busca_data=''
    try:
        busca_data=df_selic_data.loc[df_selic.index[i],'data_reuniao']
        busca_data=datetime.datetime.strptime(busca_data,'%Y-%m-%d').date()
        df_selic.loc[df_selic.index[i],'data_reuniao']=busca_data

    except:
        #### será definida um expectativa para a data.
        # busca_data=df_selic_data.loc[df_selic.index[i],'data_reuniao']
        # df_selic.loc[df_selic.index[i],'data_reuniao']=busca_data
        continue

# df_selic['data_reuniao']=df_selic.apply(lambda x: datetime.datetime.strptime(x.data_reuniao,'%Y-%m-%d').date(),axis=1)
# df_selic=df_selic[df_selic['data_reuniao']!='']
df_selic=df_selic.sort_values(by=['Data','data_reuniao','baseCalculo'],ascending=[False,True,False])

'''Base de cálculo para as estatísticas
 baseada no prazo de validade das expectativas
 informadas pelas instituições informantes:
  - 0: uso das expectativas mais recentes informadas
   pelas instituições participantes a partir do
   30º dia anterior à data de cálculo das estatísticas 
  - 1: uso das expectativas mais recentes informadas
   pelas instituições participantes a partir do
    4º dia útil anterior à data de cálculo das estatísticas'''

df_selic.head(20)

,Data,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo,data_reuniao
Reuniao,,,,,,,,,
R1/2023,2023-01-27,13.75,13.75,0.00,13.75,13.75,78,1,2023-02-01
R1/2023,2023-01-27,13.75,13.75,0.00,13.75,13.75,126,0,2023-02-01
R2/2023,2023-01-27,13.75,13.75,0.00,13.75,13.75,78,1,2023-03-22
R2/2023,2023-01-27,13.75,13.75,0.00,13.75,13.75,126,0,2023-03-22
R3/2023,2023-01-27,13.74,13.75,0.09,13.00,13.75,78,1,2023-05-03
R3/2023,2023-01-27,13.74,13.75,0.08,13.00,13.75,126,0,2023-05-03
R4/2023,2023-01-27,13.69,13.75,0.20,12.50,13.75,78,1,2023-06-21
R4/2023,2023-01-27,13.70,13.75,0.18,12.50,13.75,126,0,2023-06-21
R5/2023,2023-01-27,13.57,13.75,0.36,12.00,13.75,78,1,2023-08-02


In [5]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_selic,x='data_reuniao',y='Mediana', color='Data')
# Gerando arquivo com o resultado
# plotly.offline.plot(fig, show_link = True,filename=f'./output/IPCA_forcast.html')
fig.show()

In [6]:
# Informada à partir do 4º dia útil anterior à data de cálculo da expectativa.
df_selic=df_selic[df_selic['baseCalculo']==1]           
# Expectativa mais atual
df_selic_ultima=df_selic[df_selic['Data']==df_selic['Data'].max()]
# df_selic_ultima['mediana_mensal']=df_selic_ultima.apply(lambda x: (1+(x.Mediana/100))**(1/12)-1,axis=1)

In [7]:
# Insere coluna com números de dias no período (entre duas datas)
df_selic_ultima.loc[df_selic_ultima.index[0],'periodo']=(df_selic_ultima.loc[df_selic_ultima.index[0],'data_reuniao']-datetime.datetime.today().date()).days
# Define número índice de ref. 100 na data de hoje.
selic_atual=''
while type(selic_atual)!=float:
    try:
        selic_atual=float(input('Qual é a SELIC atual?(\d\d.d\d\)'))
    except:
        continue

C:\Users\bruno.bfn\AppData\Local\Temp\ipykernel_13924\1858555858.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
# Cria num_indice inicial atribuindo 100 para a data de hoje
## Pode acontecer de demora para atualização do json do bc, gerando o primeiro período negativo, mas não tem problema
df_selic_ultima.loc[df_selic_ultima.index[0],'num_indice']=100*(1+selic_atual/100)**(df_selic_ultima.loc[df_selic_ultima.index[0],'periodo']/365)

# num_indice_f=num_indice_i*(1+taxa)**(periodo)
for i in range(1,len(df_selic_ultima)):
    df_selic_ultima.loc[df_selic_ultima.index[i],'periodo']=(df_selic_ultima.loc[df_selic_ultima.index[i],'data_reuniao']-df_selic_ultima.loc[df_selic_ultima.index[i-1],'data_reuniao']).days
    df_selic_ultima.loc[df_selic_ultima.index[i],'num_indice']=df_selic_ultima.loc[df_selic_ultima.index[i-1],'num_indice']*(1+df_selic_ultima.loc[df_selic_ultima.index[i-1],'Mediana']/100)**(df_selic_ultima.loc[df_selic_ultima.index[i],'periodo']/365)
df_selic_ultima.to_csv('./output/selic_ultima_expectativa.csv',float_format='%.2f')


C:\Users\bruno.bfn\AppData\Local\Temp\ipykernel_13924\2648126627.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
df_selic_ultima

,Data,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo,data_reuniao,periodo,num_indice
Reuniao,,,,,,,,,,,
R1/2023,2023-01-27,13.75,13.75,0.00,13.75,13.75,78,1,2023-02-01,2.00,100.07
R2/2023,2023-01-27,13.75,13.75,0.00,13.75,13.75,78,1,2023-03-22,49.00,101.82
R3/2023,2023-01-27,13.74,13.75,0.09,13.00,13.75,78,1,2023-05-03,42.00,103.34
R4/2023,2023-01-27,13.69,13.75,0.20,12.50,13.75,78,1,2023-06-21,49.00,105.14
R5/2023,2023-01-27,13.57,13.75,0.36,12.00,13.75,78,1,2023-08-02,42.00,106.71
R6/2023,2023-01-27,13.34,13.62,0.56,11.50,13.75,78,1,2023-09-20,49.00,108.57
R7/2023,2023-01-27,12.99,13.00,0.72,11.00,13.75,78,1,2023-11-01,42.00,110.18
R8/2023,2023-01-27,12.62,12.50,0.92,10.50,13.75,78,1,2023-12-13,42.00,111.74
R1/2024,2023-01-27,12.18,12.25,1.03,9.75,13.75,73,1,2024-01-31,49.00,113.52


In [10]:
#### Função que calcula a SELIC do período entre data atual e data final
def calc_selic_periodo(df,data_f):
    # DF é um dataframe que contenha a culuna com num_indice
    
    # Utiliza a taxa digitada real vigente
    if data_f<=df.loc[df.index[0],'data_reuniao']:
        n_dias=(data_f-datetime.datetime.today().date()).days
        return 100*(1+selic_atual/100)**(n_dias/365)
    
    # Utiliza o número índice anterior + variação dos dias dentro do período
    else:
        # Último número índice
        df=df[df['data_reuniao']<=data_f]
        n_indice=df.loc[df.index[-1],'num_indice']
        taxa=df.loc[df.index[-1],'Mediana']
        n_dias=(data_f-df.loc[df.index[-1],'data_reuniao']).days
        return n_indice*(1+taxa/100)**(n_dias/365)


In [11]:
calc_selic_periodo(df_selic_ultima,datetime.datetime(2022,10,25).date())

96.63417035825701

In [12]:
df_monthly=pd.read_csv('./output/monthly_data_df.csv')
df_monthly=df_monthly.drop(columns=['Unnamed: 0'])

# Parses
df_monthly['Data']=df_monthly.apply(lambda x: datetime.datetime.strptime(x.Data,'%Y-%m-%d').date(),axis=1)

df_monthly['DataReferencia']=df_monthly.apply(lambda x: datetime.datetime.strptime(x.DataReferencia,'%m/%Y').date(),axis=1)
delta = relativedelta(months=1)
delta_d = timedelta(days=1)
df_monthly['DataReferencia']=df_monthly.apply(lambda x: x.DataReferencia+delta-delta_d,axis=1)

df_monthly=df_monthly.sort_values(by=['Data','DataReferencia','numeroRespondentes'],ascending=[False,True,False])
# df_monthly=df_selic[df_selic['baseCalculo']==1]           # não sei qual é a diferença da base de cálculo.
df_monthly.head(20)

,Indicador,Data,DataReferencia,Media,Mediana,Minimo,Maximo,numeroRespondentes
419,Taxa de desocupação,2023-01-27,2022-12-31,8.09,8.00,7.50,8.91,56
418,Taxa de desocupação,2023-01-27,2022-12-31,8.06,8.00,7.50,8.91,31
403,IPCA,2023-01-27,2023-01-31,0.55,0.55,0.15,0.76,140
405,Câmbio,2023-01-27,2023-01-31,5.20,5.20,5.00,5.42,110
402,IPCA,2023-01-27,2023-01-31,0.55,0.56,0.43,0.62,104
417,IPCA Administrados,2023-01-27,2023-01-31,0.64,0.64,0.02,1.18,86
409,IPCA Livres,2023-01-27,2023-01-31,0.50,0.50,0.33,0.69,81
411,IPCA Serviços,2023-01-27,2023-01-31,0.50,0.52,0.21,0.75,71
413,IPCA Bens industrializados,2023-01-27,2023-01-31,0.38,0.37,0.00,0.70,71
415,IPCA Alimentação no domicílio,2023-01-27,2023-01-31,0.67,0.65,0.00,1.54,71


In [13]:
df_monthly['Indicador'].unique()

array(['Taxa de desocupação', 'IPCA', 'Câmbio', 'IPCA Administrados',
       'IPCA Livres', 'IPCA Serviços', 'IPCA Bens industrializados',
       'IPCA Alimentação no domicílio', 'IGP-M'], dtype=object)

In [14]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='IPCA'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/IPCA_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='IGP-M'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/IGPM_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='Câmbio'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/cambio_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='Taxa de desocupação'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/tx_desocupacao_forcast.html')
# fig.show()

'./output/tx_desocupacao_forcast.html'

In [15]:
# Expectativa mais atual para IPCA
df_ipca=df_monthly[df_monthly['Indicador']=='IPCA']

data_previsao=df_ipca['Data'].max()
df_ipca_ultima=df_ipca[df_ipca['Data']==data_previsao]
df_ipca_ultima=df_ipca_ultima.groupby(['DataReferencia']).mean() # adotado média 

df_ipca_ultima['cumsum_mediana'] = df_ipca_ultima['Mediana'].cumsum()
#cumsum()-> soma acumulada de uma lista ou array termo a termo
#groupby -> delimita em grupos separados para análise
# df['Adjusted Quantity'] = df.groupby('ação_ref')['Adjusted Quantity'].cumsum()

In [16]:
# Insere coluna com números de dias no período (entre duas datas)
df_ipca_ultima.loc[df_ipca_ultima.index[0],'periodo']=(df_ipca_ultima.index[0]-datetime.datetime.today().date()).days

# Define número índice de ref. 100 na data de hoje.
df_ipca_ultima.loc[df_ipca_ultima.index[0],'num_indice']=100*(1+df_ipca_ultima.loc[df_ipca_ultima.index[0],'Mediana']/100)**(df_ipca_ultima.loc[df_ipca_ultima.index[0],'periodo']/30)
for i in range(1,len(df_ipca_ultima)):
    df_ipca_ultima.loc[df_ipca_ultima.index[i],'periodo']=(df_ipca_ultima.index[i]-df_ipca_ultima.index[i-1]).days
    df_ipca_ultima.loc[df_ipca_ultima.index[i],'num_indice']=df_ipca_ultima.loc[df_ipca_ultima.index[i-1],'num_indice']*(1+df_ipca_ultima.loc[df_ipca_ultima.index[i],'Mediana']/100)**(df_ipca_ultima.loc[df_ipca_ultima.index[i],'periodo']/30)
df_ipca_ultima.to_csv('./output/ipca_ultima_expectativa.csv',float_format='%.4f')
df_ipca_ultima

,Media,Mediana,Minimo,Maximo,numeroRespondentes,cumsum_mediana,periodo,num_indice
DataReferencia,,,,,,,,
2023-01-31,0.55,0.56,0.29,0.69,122.00,0.56,1.00,100.02
2023-02-28,0.79,0.80,0.45,1.17,123.00,1.35,28.00,100.77
2023-03-31,0.65,0.59,0.20,1.42,123.00,1.94,31.00,101.38
2023-04-30,0.59,0.57,0.17,1.27,122.00,2.51,30.00,101.96
2023-05-31,0.39,0.38,0.14,0.86,121.00,2.89,31.00,102.35
2023-06-30,0.37,0.35,0.13,0.70,121.00,3.24,30.00,102.71
2023-07-31,0.31,0.30,-0.09,0.60,121.00,3.54,31.00,103.03
2023-08-31,0.24,0.25,-0.01,0.58,121.00,3.79,31.00,103.30
2023-09-30,0.32,0.32,-0.05,0.52,121.00,4.11,30.00,103.63


In [17]:
# from matplotlib.pyplot import figure
# figure(figsize=(10, 5), dpi=120)
# plt.plot(df_ipca_ultima.index, df_ipca_ultima['Mediana'], label=f'ipca_expectaiva_{data_previsao}')
# plt.xticks(rotation=90)
# plt.legend()
# plt.show()


In [18]:
#### Função que calcula a IPCA do período entre data hoje e data final
def calc_ipca_periodo(df,data_f):
    # DF é um dataframe que contenha a coluna com num_indice
    
    # Utiliza a taxa digitada real vigente
    if data_f<=df.index[0]:
        # print('primeiro mês')
        n_dias=(data_f-datetime.datetime.today().date()).days
        # print(n_dias)
        return 100*(1+df.loc[df.index[0],'Mediana']/100)**(n_dias/30)
    
    # Não calcula num_indice para o último termo, pois já está tabelado. 
    elif data_f==df.index[-1]:
        return df.loc[df.index[-1],'num_indice']

    # Utiliza o número índice anterior + variação dos dias dentro do período
    else:
        # Último número índice
        df_anterior=df[df.index<=data_f]
        # print(df_anterior)
        n_indice=df_anterior.loc[df_anterior.index[-1],'num_indice']
        #busca o 1º termo depois do corte da df_anterior
        taxa=df.loc[df.index[len(df_anterior)],'Mediana']    
        # print(f'taxa:{taxa}')
        n_dias=(data_f-df_anterior.index[-1]).days
        # print(f'n_dias:{n_dias}')
        return n_indice*(1+taxa/100)**(n_dias/30)

In [19]:
calc_ipca_periodo(df_ipca_ultima,datetime.datetime(2022,11,30).date())

98.88092876097939

In [20]:
df_ipca_ultima

,Media,Mediana,Minimo,Maximo,numeroRespondentes,cumsum_mediana,periodo,num_indice
DataReferencia,,,,,,,,
2023-01-31,0.55,0.56,0.29,0.69,122.00,0.56,1.00,100.02
2023-02-28,0.79,0.80,0.45,1.17,123.00,1.35,28.00,100.77
2023-03-31,0.65,0.59,0.20,1.42,123.00,1.94,31.00,101.38
2023-04-30,0.59,0.57,0.17,1.27,122.00,2.51,30.00,101.96
2023-05-31,0.39,0.38,0.14,0.86,121.00,2.89,31.00,102.35
2023-06-30,0.37,0.35,0.13,0.70,121.00,3.24,30.00,102.71
2023-07-31,0.31,0.30,-0.09,0.60,121.00,3.54,31.00,103.03
2023-08-31,0.24,0.25,-0.01,0.58,121.00,3.79,31.00,103.30
2023-09-30,0.32,0.32,-0.05,0.52,121.00,4.11,30.00,103.63


In [21]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_ipca_ultima,x=df_ipca_ultima.index,y=['cumsum_mediana'])
# Gerando arquivo com o resultado
# plotly.offline.plot(fig, show_link = True,filename=f'./output/tx_desocupacao_forcast.html')
fig.show()

In [22]:
# Relação de datas até o mínimo entre os dataframes, para evitar erros
lista_datas_todos_dias=pd.date_range(start=datetime.datetime.today().date(),end=min(df_selic_ultima['data_reuniao'].max(),df_ipca_ultima.index.max()))
df_juros_real=pd.DataFrame(index=lista_datas_todos_dias,columns=['num_indice_selic','num_indice_ipca'],dtype='float64')
for i in range(0,len(df_juros_real)):
    df_juros_real.loc[df_juros_real.index[i],'num_indice_selic']=calc_selic_periodo(df_selic_ultima,df_juros_real.index[i].date())
    df_juros_real.loc[df_juros_real.index[i],'num_indice_ipca']=calc_ipca_periodo(df_ipca_ultima,df_juros_real.index[i].date())
df_juros_real

,num_indice_selic,num_indice_ipca
2023-01-30,100.00,100.00
2023-01-31,100.04,100.02
2023-02-01,100.07,100.05
2023-02-02,100.11,100.07
2023-02-03,100.14,100.10
...,...,...
2024-12-11,124.28,108.89
2024-12-12,124.31,108.90
2024-12-13,124.34,108.92
2024-12-14,124.37,108.94


In [23]:
df_ipca_ultima

,Media,Mediana,Minimo,Maximo,numeroRespondentes,cumsum_mediana,periodo,num_indice
DataReferencia,,,,,,,,
2023-01-31,0.55,0.56,0.29,0.69,122.00,0.56,1.00,100.02
2023-02-28,0.79,0.80,0.45,1.17,123.00,1.35,28.00,100.77
2023-03-31,0.65,0.59,0.20,1.42,123.00,1.94,31.00,101.38
2023-04-30,0.59,0.57,0.17,1.27,122.00,2.51,30.00,101.96
2023-05-31,0.39,0.38,0.14,0.86,121.00,2.89,31.00,102.35
2023-06-30,0.37,0.35,0.13,0.70,121.00,3.24,30.00,102.71
2023-07-31,0.31,0.30,-0.09,0.60,121.00,3.54,31.00,103.03
2023-08-31,0.24,0.25,-0.01,0.58,121.00,3.79,31.00,103.30
2023-09-30,0.32,0.32,-0.05,0.52,121.00,4.11,30.00,103.63


In [24]:
# Não utilizada
def area(tipo,date):
    #tipo={selic,ipca}
    area=0
    df=df_juros_real[df_juros_real.index<=date]
    if (tipo=='selic' or tipo=='ipca'):
        for i in range(0,len(df)):
            area=area+1*df.loc[df.index[i],f'num_indice_{tipo}']
                
        return area
    else:
        print('Erro na escolha do tipo (selic ou ipca)')
    return 0

In [25]:
# df_juros_real['area_selic']=0
# for i in range(0,len(df_juros_real)):
#     # print(area('selic',df_juros_real.index[i]))
#     df_juros_real.loc[df_juros_real.index[i], 'area_selic']=area('selic',df_juros_real.index[i])
#     df_juros_real.loc[df_juros_real.index[i], 'area_ipca']=area('ipca',df_juros_real.index[i])

# df_juros_real

In [26]:
df_juros_real['dif_num_indice']=df_juros_real.apply(lambda x: x.num_indice_selic-x.num_indice_ipca+100,axis=1)
# df_juros_real['dif_area']=df_juros_real.apply(lambda x: x.area_selic-x.area_ipca,axis=1)
# df_juros_real['dif_area_por_dia']=df_juros_real.apply(lambda x: x.dif_area/(x.index-))
df_juros_real.to_csv('./output/num_indice_selic_ipca_diferenca.csv',float_format='%.2f')
df_juros_real


,num_indice_selic,num_indice_ipca,dif_num_indice
2023-01-30,100.00,100.00,100.00
2023-01-31,100.04,100.02,100.02
2023-02-01,100.07,100.05,100.03
2023-02-02,100.11,100.07,100.03
2023-02-03,100.14,100.10,100.04
...,...,...,...
2024-12-11,124.28,108.89,115.39
2024-12-12,124.31,108.90,115.40
2024-12-13,124.34,108.92,115.42
2024-12-14,124.37,108.94,115.43


In [27]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_juros_real,x=df_juros_real.index,y=['num_indice_selic','num_indice_ipca','dif_num_indice'])
# Gerando arquivo com o resultado
# plotly.offline.plot(fig, show_link = True,filename=f'./output/IPCA_forcast.html')
fig.show()

In [28]:
from time import strptime
from pyxirr import xirr
# calculo da IRR
def cal_irr(x,date,tipo):                #DF refer^rencia e a data de análise

    if date==x.index[0]:
        return (0,0,0,0)
    
    dates=[x.index[0],date]
    
    # tipo pode assumir ('selic' ou 'pre')
    if tipo=='selic':
        amounts=[-x.loc[x.index[0],'dif_num_indice'],x.loc[date,'dif_num_indice']]
    elif tipo=='pre_1':
        amounts=[-x.loc[x.index[0],'dif_num_indice_pre_1'],x.loc[date,'dif_num_indice_pre_1']]
    elif tipo=='pre_2':
        amounts=[-x.loc[x.index[0],'dif_num_indice_pre_2'],x.loc[date,'dif_num_indice_pre_2']]
            
    else:
        print('Erro de tipo diferente de "selic" ou "pre"') 
    # date=datetime.datetime.strptime(date,'%Y-%m-%d').date()
    # x=x[x.index==x.index[0] | x.index.date()==date]
    # dates=x.index
    # amounts=x['dif_num_indice']
    t_aa=xirr(dates, amounts)
    t_am=(1+t_aa)**(1/12)-1
    # print('TII a.a.(%): ',t_aa*100)
    # print('TII a.m.(%): ',t_am*100)
    # print('Soma dos valores: ',-x['Valor'].sum())
    # print(dates)
    # print(amounts)
    return (t_aa*100,t_am*100,dates,amounts)


In [29]:
for i in range(0,len(df_juros_real)):
    df_juros_real.loc[df_juros_real.index[i],'juros_real_acum_from_now']=cal_irr(df_juros_real,df_juros_real.index[i],'selic')[0]
    df_juros_real.loc[df_juros_real.index[i],'juros_real_am']=cal_irr(df_juros_real,df_juros_real.index[i],'selic')[1]

df_juros_real.head(30)

,num_indice_selic,num_indice_ipca,dif_num_indice,juros_real_acum_from_now,juros_real_am
2023-01-30,100.00,100.00,100.00,0.00,0.00
2023-01-31,100.04,100.02,100.02,6.34,0.51
2023-02-01,100.07,100.05,100.03,4.78,0.39
2023-02-02,100.11,100.07,100.03,4.27,0.35
2023-02-03,100.14,100.10,100.04,4.01,0.33
2023-02-04,100.18,100.12,100.05,3.86,0.32
2023-02-05,100.21,100.15,100.06,3.76,0.31
2023-02-06,100.25,100.18,100.07,3.68,0.30
2023-02-07,100.28,100.20,100.08,3.63,0.30
2023-02-08,100.32,100.23,100.09,3.59,0.29


In [30]:
for i in range(0,len(df_juros_real)):
    # Primeiro dia do mês
    inicio_mes=df_juros_real.index[i]-timedelta(df_juros_real.index[i].day-1)
    if inicio_mes<=df_juros_real.index[0]:
        inicio_mes=df_juros_real.index[0]
    # print(f'Início do mês: {inicio_mes} - até: {df_juros_real.index[i]}')
    # Slice do DF do primeiro dia do mês até o dia da análise
    try:
        df_juros_real_i=df_juros_real[df_juros_real.index>=inicio_mes]
        df_juros_real_i=df_juros_real_i[df_juros_real_i.index<=df_juros_real.index[i]]
        # print(df_juros_real_i)
        df_juros_real.loc[df_juros_real.index[i],'juros_real_mesal']=cal_irr(df_juros_real_i,df_juros_real.index[i],'selic')[0]
    except:
        continue
df_juros_real.head(30)

,num_indice_selic,num_indice_ipca,dif_num_indice,juros_real_acum_from_now,juros_real_am,juros_real_mesal
2023-01-30,100.00,100.00,100.00,0.00,0.00,0.00
2023-01-31,100.04,100.02,100.02,6.34,0.51,6.34
2023-02-01,100.07,100.05,100.03,4.78,0.39,0.00
2023-02-02,100.11,100.07,100.03,4.27,0.35,3.24
2023-02-03,100.14,100.10,100.04,4.01,0.33,3.25
2023-02-04,100.18,100.12,100.05,3.86,0.32,3.25
2023-02-05,100.21,100.15,100.06,3.76,0.31,3.25
2023-02-06,100.25,100.18,100.07,3.68,0.30,3.25
2023-02-07,100.28,100.20,100.08,3.63,0.30,3.25
2023-02-08,100.32,100.23,100.09,3.59,0.29,3.25


In [31]:
def calc_pre_periodo(data_f,taxa_pre):
    n_dias=(data_f-datetime.datetime.today().date()).days
    return 100*(1+taxa_pre/100)**(n_dias/365)

In [32]:
calc_pre_periodo(datetime.datetime(2022,10,29).date(),12)

97.15373861062251

In [33]:
# Gera num_indice_taxa_pre-fixa
taxa_pre_1=12.85
taxa_pre_2=13.19
for i in range(0,len(df_juros_real)):
    df_juros_real.loc[df_juros_real.index[i],f'num_indice_pre_1']=calc_pre_periodo(df_juros_real.index[i].date(),taxa_pre_1)
    df_juros_real.loc[df_juros_real.index[i],f'num_indice_pre_2']=calc_pre_periodo(df_juros_real.index[i].date(),taxa_pre_2)

df_juros_real


,num_indice_selic,num_indice_ipca,dif_num_indice,juros_real_acum_from_now,juros_real_am,juros_real_mesal,num_indice_pre_1,num_indice_pre_2
2023-01-30,100.00,100.00,100.00,0.00,0.00,0.00,100.00,100.00
2023-01-31,100.04,100.02,100.02,6.34,0.51,6.34,100.03,100.03
2023-02-01,100.07,100.05,100.03,4.78,0.39,0.00,100.07,100.07
2023-02-02,100.11,100.07,100.03,4.27,0.35,3.24,100.10,100.10
2023-02-03,100.14,100.10,100.04,4.01,0.33,3.25,100.13,100.14
...,...,...,...,...,...,...,...,...
2024-12-11,124.28,108.89,115.39,7.97,0.64,4.82,125.30,126.01
2024-12-12,124.31,108.90,115.40,7.97,0.64,4.82,125.34,126.05
2024-12-13,124.34,108.92,115.42,7.97,0.64,4.82,125.38,126.09
2024-12-14,124.37,108.94,115.43,7.96,0.64,4.82,125.43,126.13


In [34]:
# Agora que tenho num_indice_pre -> realizando os cálculos análogamente ao num_indice_selic
df_juros_real['dif_num_indice_pre_1']=df_juros_real.apply(lambda x: x.num_indice_pre_1-x.num_indice_ipca+100,axis=1)
df_juros_real['dif_num_indice_pre_2']=df_juros_real.apply(lambda x: x.num_indice_pre_2-x.num_indice_ipca+100,axis=1)


for i in range(0,len(df_juros_real)):
    df_juros_real.loc[df_juros_real.index[i],f'juros_real_acum_from_now_pre_{taxa_pre_1}%']=cal_irr(df_juros_real,df_juros_real.index[i],'pre_1')[0]
    df_juros_real.loc[df_juros_real.index[i],f'juros_real_acum_from_now_pre_{taxa_pre_2}%']=cal_irr(df_juros_real,df_juros_real.index[i],'pre_2')[0]

df_juros_real

,num_indice_selic,num_indice_ipca,dif_num_indice,juros_real_acum_from_now,juros_real_am,juros_real_mesal,num_indice_pre_1,num_indice_pre_2,dif_num_indice_pre_1,dif_num_indice_pre_2,juros_real_acum_from_now_pre_12.85%,juros_real_acum_from_now_pre_13.19%
2023-01-30,100.00,100.00,100.00,0.00,0.00,0.00,100.00,100.00,100.00,100.00,0.00,0.00
2023-01-31,100.04,100.02,100.02,6.34,0.51,6.34,100.03,100.03,100.01,100.02,5.50,5.82
2023-02-01,100.07,100.05,100.03,4.78,0.39,0.00,100.07,100.07,100.02,100.02,3.95,4.27
2023-02-02,100.11,100.07,100.03,4.27,0.35,3.24,100.10,100.10,100.03,100.03,3.44,3.75
2023-02-03,100.14,100.10,100.04,4.01,0.33,3.25,100.13,100.14,100.03,100.04,3.19,3.50
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-11,124.28,108.89,115.39,7.97,0.64,4.82,125.30,126.01,116.42,117.12,8.49,8.84
2024-12-12,124.31,108.90,115.40,7.97,0.64,4.82,125.34,126.05,116.44,117.15,8.49,8.84
2024-12-13,124.34,108.92,115.42,7.97,0.64,4.82,125.38,126.09,116.46,117.17,8.49,8.84
2024-12-14,124.37,108.94,115.43,7.96,0.64,4.82,125.43,126.13,116.49,117.20,8.48,8.84


In [35]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_juros_real,x=df_juros_real.index,y=['juros_real_acum_from_now','juros_real_mesal',f'juros_real_acum_from_now_pre_{taxa_pre_1}%',f'juros_real_acum_from_now_pre_{taxa_pre_2}%'])
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/Juros_real_forcast.html', auto_open=False)
fig.show()
